# Ollama 

In this section we will use ollama to run an LLM locally on a CPU to replace GPT on the OPENAI API

We can use different models in Ollama as we can check in https://github.com/ollama/ollama

In [1]:
from openai import OpenAI

In [14]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama'
)

In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [6]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [20]:

def llm(prompt):
    response = client.chat.completions.create(
        model='llama3', #phi3
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [16]:
llm('write that this is a test')

' This is a test.\n\n\nIn conducting various examinener tests, one of the first steps often involves writing or entering instructions into an automated testing system to confirm its readiness for executing pre-designed scenarios aimed at evaluating certain aspects like functionality, reliability, and performance within software development environments.'

In [17]:
print(_)

 This is a test.


In conducting various examinener tests, one of the first steps often involves writing or entering instructions into an automated testing system to confirm its readiness for executing pre-designed scenarios aimed at evaluating certain aspects like functionality, reliability, and performance within software development environments.


**TEST**

This message is just a test. I'm here to help, but for now, I'll just sit here and wait for further instructions. Go ahead and give me something to do!

In [10]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [21]:
rag('I just discovered the couse. Can i still join?')

'Based on the context, I can see that we\'re talking about a course where you can still join even after the start date! As per the FAQ section, "Yes, even if you don\'t register, you\'re still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don\'t leave everything for the last minute."\n\nSo, to answer your question: Yes, you can still join the course!'

In [22]:
print(_)

Based on the context, I can see that we're talking about a course where you can still join even after the start date! As per the FAQ section, "Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

So, to answer your question: Yes, you can still join the course!
